### Import the necessary program libraries

In [1]:
import pandas as pd
from classes import KMeansClusterMain, BaseLoader
import plotly.express as px
from sqlalchemy import create_engine
from settings import mariadb_string, mariadb_config
from funcs import get_density
from pathlib import Path

### Automate of receiving pics

In [2]:

loader = BaseLoader(mariadb_config)
symbols = loader.select("select symbol from stocks;")

data_select_query = """select 
	*
from
	stock_price s
where
	s.symbol = '{}'
group BY
	s.`date`
order by
	s.`date` asc;"""

for item in symbols:
    df_stock = pd.read_sql(data_select_query.format(item[0]), create_engine(mariadb_string))
    df_stock["date"] = pd.to_datetime(df_stock["date"])
    
    # Compute the percentage change
    df_stock["perc_change"] = ((df_stock["open"] + df_stock["high"] + df_stock["low"] + df_stock["close"]) / 3).pct_change()
    df_stock.dropna(inplace=True)
    df_stock = df_stock[df_stock['perc_change'] != 0.0]
    
    # Here comes KMeans
    datapoints = df_stock["perc_change"].to_list()
    kmeans = KMeansClusterMain(3)
    kmeans.fit(datapoints) 
    
    # Assign the clusters
    df_stock["cluster"] = kmeans.labels
    df_stock["cluster"] = pd.to_numeric(df_stock["cluster"])
    
    # Get the density for the colormap
    df_stock["density"] = df_stock["cluster"].rolling(21).apply(get_density, kwargs={'orig_density': kmeans.centroids})
    
    # cluster_str needed for kmeans volatility cluster graph.
    df_stock["cluster_str"] = df_stock.cluster.astype(str)
    df_stock['perc_change'] = df_stock['perc_change'] * 100
    
    # Calculate the density
    describe_ser = df_stock["density"].describe()
    upper_limit = describe_ser["mean"] + (describe_ser["std"] * 2)
    lower_limit = describe_ser["mean"] - (describe_ser["std"] * 2)
    
    # Show the density
    fig = px.line(df_stock, x="date", y="density", title="Stock on Daily Basis",
                      labels={"date": "Date", "density": "Cluster Aufsummierung"})
    
    fig.add_hline(y=upper_limit, line_dash="dash", line_color="red", annotation_text=f"Grenze bei {round(upper_limit,4)}")
    fig.add_hline(y=lower_limit, line_dash="dash", line_color="red", annotation_text=f"Grenze bei {round(lower_limit,4)}")
    
    
    fig.write_image(str(Path.cwd().joinpath("images", item[0] + "_kmeans_analyse")) + ".png")
    print(f"Image for {item[0]} saved!")

print("All good!")

Image for ADS.DEX saved!
Image for AMZN saved!
Image for BKNG saved!
Image for DASH saved!
Image for FNTN.DE saved!
Image for HD saved!
Image for LOW saved!
Image for MCD saved!
Image for META saved!
Image for NKE saved!
Image for NRDBY saved!
Image for PAH3.DE saved!
Image for SBUX saved!
Image for SY1.DE saved!
Image for TJX saved!
Image for TSLA saved!
All good!


In [5]:
item

('MBG.DE',)